In [1]:
%run -i ../python/common.py
publish=False

if not publish:
    # cleanup any old state
    # demke - fill in as we see what state gets generated in this page.
    bashCmds('''[[ -d mydir ]] && rm -rf mydir
    #''')
else:
    bashCmds('''rm -rf ~/*''')
    
closeAllOpenTtySessions()



In [2]:
appdir=os.getenv('HOME')
appdir=appdir + "/sync"
output = runTermCmd("[[ -d " + appdir + " ]] &&  rm -rf "+ appdir + 
             ";cp -r ../src/sync " + appdir)

bash = BashSession(cwd=appdir)

In [3]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/spinlock.h", 
    lang="", 
    number=True,
    title="<b>spinlock.c - Basic spinlock using C11 atomics.</b>",
    h="100%", 
    w="100%"
) + '</font>'))

<font size="1.2rem"><b>spinlock.c - Basic spinlock using C11 atomics.</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: #include <stdatomic.h>
1: 
2: typedef struct spinlock_s {
3:     atomic_flag locked;
4: } spinlock_t; 
5: 
6: static inline void spinlock_init(spinlock_t *l) {
7:     atomic_flag_clear( &l->locked );
8: }
9: 
10: static inline void spinlock_acquire(spinlock_t *l)
11: {
12:     while(atomic_flag_test_and_set( &l->locked )) { };
13: }
14: 
15: static inline void spinlock_release(spinlock_t *l)
16: {
17:     atomic_flag_clear( &l->locked );
18: }

```


</div>
</font>

In [4]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/counter_spinlock.c", 
    lang="", 
    number=True,
    title="<b>counter_spinlock.c: Shared counter example using test-and-set spinlock to protect access to counter variable.</b>",
    h="100%", 
    w="100%"
) + '</font>'))

<font size="1.2rem"><b>counter_spinlock.c: Shared counter example using test-and-set spinlock to protect access to counter variable.</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: #include <pthread.h>
1: #include <stdlib.h>
2: #include <sys/types.h>
3: #include <unistd.h>
4: #include <string.h>
5: #include <stdio.h>
6: #include "spinlock.h"
7: 
8: spinlock_t splock;
9: volatile long shared_counter;
10: 
11: static void *increment_thread(void *arg)
12: {
13:     long niters = (long)arg;
14:     for (int i = 0; i < niters; i++) {
15:         spinlock_acquire(&splock);
16:         shared_counter++;
17:         spinlock_release(&splock);
18:     }
19: 
20:     return (void *)0;
21: }
22: 
23: static void *decrement_thread(void *arg)
24: {
25:     long niters = (long)arg;
26: 
27:     for (int i = 0; i < niters; i++) {
28:         spinlock_acquire(&splock);
29:         shared_counter--;
30:         spinlock_release(&splock);
31:     }
32: 
33:     return (void *)0;
34: }
35: 
36: int main(int argc, char **argv)
37: {
38:     pthread_t *tids;
39:     long niters;
40:     int nthreads, i;
41:     
42:     if (argc != 3) {
43:         fprintf(stderr,"Usage: %s <num_threads> <num_iters>\n",argv[0]);
44:         return 1;
45:     }
46: 
47:     nthreads = atoi(argv[1]);
48:     niters = atoi(argv[2]);
49:     shared_counter = 0;
50:     spinlock_init( &splock );
51:     
52:     printf("Main thread: Beginning test with %d threads\n", nthreads);
53:     
54:     tids = (pthread_t *)malloc(nthreads * sizeof(pthread_t));
55:     
56:     /* We create the same number of increment and decrement threads, each doing the same number of iterations. 
57:      * When all threads have completed, we expect the final value of the shared counter to be the same as its
58:      * initial value (i.e., 0).
59:      */
60:     for (i = 0; i < nthreads; i+=2) {
61:         (void)pthread_create(&tids[i], NULL, increment_thread, (void *)niters );
62:         (void)pthread_create(&tids[i+1], NULL, decrement_thread, (void *)niters );
63:     }
64:     
65:     /* Wait for child threads to finish */
66:     for (i = 0; i < nthreads; i+=2) {
67:         pthread_join(tids[i], NULL);
68:         pthread_join(tids[i+1], NULL);
69:     }
70:     
71:     printf("Main thread: Final value of shared counter is %ld\n", shared_counter);
72: 
73:     return 0;
74: }

```


</div>
</font>

In [5]:
bash.runNoOutput("[[ -a counter_spinlock ]] && rm counter_spinlock;make counter_spinlock")
cmds = '''./counter_spinlock 2 20000
./counter_spinlock 2 100000
./counter_spinlock 4 100000
./counter_spinlock 4 100000'''

bash.run(cmds)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [6]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/ticketlock.h", 
    lang="", 
    number=True,
    title="<b>C11 implementation of a ticket lock</b>",
    h="100%", 
    w="100%"
) + '</font>'))

<font size="1.2rem"><b>C11 implementation of a ticket lock</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: #include <stdatomic.h>
1: 
2: typedef struct ticketlock_s {
3:     atomic_ulong next_ticket;
4:     atomic_ulong now_serving;
5: } ticketlock_t; 
6: 
7: static inline void ticketlock_init(ticketlock_t *l) {
8:     l->next_ticket = 0;
9:     l->now_serving = 0;
10: }
11: 
12: static inline void ticketlock_acquire(ticketlock_t *l)
13: {
14:     unsigned long my_ticket = atomic_fetch_add(&l->next_ticket, 1);
15:     while(my_ticket != l->now_serving) { };
16: }
17: 
18: static inline void ticketlock_release(ticketlock_t *l)
19: {
20:     l->now_serving++;
21: }

```


</div>
</font>

In [7]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/counter_ticketlock.c", 
    lang="", 
    number=True,
    title="<b>counter_ticketlock.c: Shared counter example using test-and-set spinlock to protect access to counter variable.</b>",
    h="100%", 
    w="100%"
) + '</font>'))

<font size="1.2rem"><b>counter_ticketlock.c: Shared counter example using test-and-set spinlock to protect access to counter variable.</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: #include <pthread.h>
1: #include <stdlib.h>
2: #include <sys/types.h>
3: #include <unistd.h>
4: #include <string.h>
5: #include <stdio.h>
6: #include "ticketlock.h"
7: 
8: ticketlock_t tktlock;
9: volatile long shared_counter;
10: 
11: static void *increment_thread(void *arg)
12: {
13:     long niters = (long)arg;
14:     for (int i = 0; i < niters; i++) {
15:         ticketlock_acquire(&tktlock);
16:         shared_counter++;
17:         ticketlock_release(&tktlock);
18:     }
19: 
20:     return (void *)0;
21: }
22: 
23: static void *decrement_thread(void *arg)
24: {
25:     long niters = (long)arg;
26: 
27:     for (int i = 0; i < niters; i++) {
28:         ticketlock_acquire(&tktlock);
29:         shared_counter--;
30:         ticketlock_release(&tktlock);
31:     }
32: 
33:     return (void *)0;
34: }
35: 
36: int main(int argc, char **argv)
37: {
38:     pthread_t *tids;
39:     long niters;
40:     int nthreads, i;
41:     
42:     if (argc != 3) {
43:         fprintf(stderr,"Usage: %s <num_threads> <num_iters>\n",argv[0]);
44:         return 1;
45:     }
46: 
47:     nthreads = atoi(argv[1]);
48:     niters = atoi(argv[2]);
49:     shared_counter = 0;
50:     ticketlock_init( &tktlock );
51:     
52:     printf("Main thread: Beginning test with %d threads\n", nthreads);
53:     
54:     tids = (pthread_t *)malloc(nthreads * sizeof(pthread_t));
55:     
56:     /* We create the same number of increment and decrement threads, each doing the same number of iterations. 
57:      * When all threads have completed, we expect the final value of the shared counter to be the same as its
58:      * initial value (i.e., 0).
59:      */
60:     for (i = 0; i < nthreads; i+=2) {
61:         (void)pthread_create(&tids[i], NULL, increment_thread, (void *)niters );
62:         (void)pthread_create(&tids[i+1], NULL, decrement_thread, (void *)niters );
63:     }
64:     
65:     /* Wait for child threads to finish */
66:     for (i = 0; i < nthreads; i+=2) {
67:         pthread_join(tids[i], NULL);
68:         pthread_join(tids[i+1], NULL);
69:     }
70:     
71:     printf("Main thread: Final value of shared counter is %ld\n", shared_counter);
72: 
73:     return 0;
74: }

```


</div>
</font>

In [8]:
bash.runNoOutput("[[ -a counter_ticketlock ]] && rm counter_ticketlock;make counter_ticketlock")
cmds = '''./counter_ticketlock 2 20000
./counter_ticketlock 2 100000
./counter_ticketlock 4 100000
./counter_ticketlock 4 100000'''

bash.run(cmds)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [9]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/check_spinlock_fairness.c", 
    lang="", 
    number=True,
    title="<b>check_spinlock_fairness.c - Sample program to show fairness and efficiency of test-and-set spinlock implementation.</b>",
    h="100%", 
    w="100%"
) + '</font>'))

<font size="1.2rem"><b>check_spinlock_fairness.c - Sample program to show fairness and efficiency of test-and-set spinlock implementation.</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: #include <pthread.h>
1: #include <stdlib.h>
2: #include <sys/types.h>
3: #include <unistd.h>
4: #include <string.h>
5: #include <stdio.h>
6: #include <stdbool.h>
7: #include <limits.h>
8: #include "spinlock.h"
9: 
10: spinlock_t splock;
11: atomic_bool start;
12: atomic_bool done;
13: 
14: struct thread_info {
15:     pthread_t tid;
16:     unsigned iters_done;
17: };
18: 
19: static void *thread_func(void *arg)
20: {
21:     struct thread_info *my_info = (struct thread_info *)arg;
22:     unsigned i = 0;
23:     
24:     while(!start) {} ; /* Wait until parent flags that all threads have been created */ 
25:     
26:     while (!done) {
27:         spinlock_acquire(&splock);
28:         i++;
29:         spinlock_release(&splock);
30:     }
31: 
32:     my_info->iters_done = i;
33:     return (void *)0;
34: }
35: 
36: 
37: 
38: int main(int argc, char **argv)
39: {
40:     struct thread_info *tinfo;
41:     int duration;
42:     int nthreads, i;
43:     
44:     if (argc != 3) {
45:         fprintf(stderr,"Usage: %s <num_threads> <duration>\n\t where duration is the number of seconds for threads to run.\n",argv[0]);
46:         return 1;
47:     }
48: 
49:     start = false;
50:     done = false;
51:     nthreads = atoi(argv[1]);
52:     duration = atoi(argv[2]);
53:     spinlock_init( &splock );
54:     
55:     printf("Main thread: Beginning test with %d threads, running for %d seconds\n", nthreads, duration);
56:     
57:     tinfo = (struct thread_info *)calloc(nthreads, sizeof(struct thread_info));
58: 
59:     /* We create the requested number of threads, all doing the same work */
60:     for (i = 0; i < nthreads; i++) {
61:         (void)pthread_create(&tinfo[i].tid, NULL, thread_func, (void *)&tinfo[i] );
62:     }
63:     
64:     /* All threads created. Let them run concurrently for requested duration. */
65:     start = true;
66:     sleep(duration);
67:     done = true;
68:     
69:     /* Wait for child threads to finish */
70:     for (i = 0; i < nthreads; i++) {
71:         pthread_join(tinfo[i].tid, NULL);
72:     }
73: 
74:     unsigned min_iters = UINT_MAX;
75:     unsigned max_iters = 0; 
76:     unsigned tot_iters = 0;
77:     for (i = 0; i < nthreads; i++) {
78:         tot_iters += tinfo[i].iters_done;
79:         if (tinfo[i].iters_done < min_iters) {
80:             min_iters = tinfo[i].iters_done;
81:         }
82:         if (tinfo[i].iters_done > max_iters) {
83:             max_iters = tinfo[i].iters_done;
84:         }
85:         printf("Thread %d completed %u iters\n", i, tinfo[i].iters_done);
86:     }
87:     
88:     unsigned diff = max_iters - min_iters;
89:     printf("\nTOTAL: %u iterations completed.\n", tot_iters);
90:     printf("Unfairness: Most productive thread completed %u iters, least productive thread completed %u iters\n", max_iters, min_iters);
91:     printf("Difference is %u iters\n", diff); 
92: 
93:     return 0;
94: }

```


</div>
</font>

In [10]:
bash.runNoOutput("[[ -a check_spinlock_fairness ]] && rm check_spinlock_fairness;make check_spinlock_fairness")
bash.run("./check_spinlock_fairness 6 5")

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [11]:
bash.runNoOutput("[[ -a check_ticketlock_fairness ]] && rm check_ticketlock_fairness;make check_ticketlock_fairness")
bash.run("./check_ticketlock_fairness 6 5")

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [12]:
bash.runNoOutput("[[ -a check_mutex_fairness ]] && rm check_mutex_fairness;make check_mutex_fairness")
bash.run("./check_mutex_fairness 6 5")

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…